In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


In [6]:
train_df = pd.read_csv("/kaggle/input/playground-series-s4e1/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s4e1/test.csv")


# train = 165034행, 14열
# test = 110023행, 13열

## train이 약 test 1.5배

X_trian

In [7]:
X_train

NameError: name 'X_train' is not defined

In [ ]:
train_df

In [ ]:
train_df.shape

In [ ]:
test_df.head()

In [ ]:
test_df.shape

### id 아이디
### CustomerId 회원아이디
### Surname 성
### CreditScore 신용점수
### Geography 지역
### Gender 성별
### Age 나이
### Tenure 기간
### Balance 잔액
### NumOfProducts 상품갯수
### HasCrCard 카드보유여부 
### IsActiveMember 활성멤버여부
### EstimatedSalary 예상연봉 
### Exited 이탈여부

In [ ]:
train_df.info()

In [ ]:
train_df.isnull().sum()

## 결측치 없음

In [ ]:
train_df["Exited"].value_counts()

In [ ]:
import seaborn as sns
sns.countplot(data=train_df, x="Exited")

In [ ]:
test_df.isnull().sum()

In [ ]:
test_df.describe(include="number")

## Balance,NumOfProducts, HasCrCard, IsActiveMember 범주형으로 바꿔야할듯

In [ ]:
# cols = X_train.columns[:-1].tolist()
# cols

In [ ]:
X_train.info()

In [ ]:
h=X_train.hist(figsize=(15,15),bins=20)

# 결측치 채우기

In [8]:
train_df["Balance_nan"] = train_df["Balance"].replace(0,np.nan)
test_df["Balance_nan"] = test_df["Balance"].replace(0,np.nan)
train_df[["Balance","Balance_nan"]].head()

,Balance,Balance_nan
0,0.00,NaN
1,0.00,NaN
2,0.00,NaN
3,148882.54,148882.54
4,0.00,NaN


In [ ]:
# train_df["NumOfProducts_nan"] = train_df["NumOfProducts"].replace(0,np.nan)
# test_df["NumOfProducts_nan"] = test_df["NumOfProducts"].replace(0,np.nan)
# train_df[["NumOfProducts","NumOfProducts_nan"]].head()

In [ ]:
test_df[["Balance","Balance_nan"]].head()

In [ ]:
train_df["Balance_nan"].isnull().mean()

In [ ]:
test_df["Balance_nan"].isnull().mean()

In [ ]:
# train_df.groupby(["Exited"])["Balance","Balance_nan"].agg(["mean","median"])

In [9]:
train_df["Balance_nan"]=train_df["Balance_nan"].fillna(train_df["Balance_nan"].median())
test_df["Balance_nan"]=test_df["Balance_nan"].fillna(test_df["Balance_nan"].median())
train_df["Balance_nan"]

0         122314.39
1         122314.39
2         122314.39
3         148882.54
4         122314.39
            ...    
165029    122314.39
165030    122314.39
165031    122314.39
165032    161533.00
165033    122314.39
Name: Balance_nan, Length: 165034, dtype: float64

In [ ]:
train_df

In [ ]:
# train_df["NumOfProducts_nan"].fillna(train_df["NumOfProducts_nan"].median())
# test_df["NumOfProducts_nan"].fillna(test_df["NumOfProducts_nan"].median())

In [ ]:
test_df

In [ ]:
# fig,axes = plt.subplots(nrows=3,ncols=3,figsize=(15,15))
# for i,col_name in enumerate(cols):
#   row = i//3
#   col = i%3
#   print(i,col_name,row,col)
#   sns.distplot(df[col_name],ax=axes[row][col])

In [ ]:
# df_number = train_df.select_dtypes(exclude='object')
# sns.pairplot(df_number,hue="Exited")

In [ ]:
# y = train_df["Exited"]
# y

# 범주형 -> 숫자형으로 변경

In [10]:
sex_train_dummies = pd.get_dummies(train_df['Gender'])
sex_test_dummies = pd.get_dummies(test_df['Gender'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Gender'], axis=1, inplace=True)
test_df.drop(['Gender'], axis=1, inplace=True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

In [11]:
train_df['Geography'].value_counts()

Geography
France     94215
Spain      36213
Germany    34606
Name: count, dtype: int64

In [12]:
Geography_train_dummies = pd.get_dummies(train_df['Geography'])
Geography_test_dummies = pd.get_dummies(test_df['Geography'])

Geography_train_dummies.columns = ['France', 'Spain',"Germany"]
Geography_test_dummies.columns = ['France', 'Spain',"Germany"]
train_df.drop(['Geography'], axis=1, inplace=True)
test_df.drop(['Geography'], axis=1, inplace=True)

train_df = train_df.join(Geography_train_dummies)
test_df = test_df.join(Geography_test_dummies)

# 알고리즘 적용하기

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
df_0 = train_df[train_df["Exited"]==0]
df_1 = train_df[train_df["Exited"]==1]
df_0.shape, df_1.shape

In [ ]:
sns.distplot(df_0["NumOfProducts"])
sns.distplot(df_1["NumOfProducts"])

In [ ]:
sns.distplot(df_0["Age"])
sns.distplot(df_1["Age"])

In [13]:
train_df["Age_low"] = train_df["Age"] <= 40
train_df["Age_high"] = train_df["Age"] > 40
train_df[["Age","Age_high"]].head()

,Age,Age_high
0,33.0,False
1,33.0,False
2,40.0,False
3,34.0,False
4,33.0,False


In [14]:
test_df["Age_low"] = test_df["Age"] < 40
test_df["Age_high"] = test_df["Age"] >= 40
test_df[["Age","Age_high"]].head()

,Age,Age_high
0,23.0,False
1,46.0,True
2,34.0,False
3,36.0,False
4,38.0,False


In [ ]:
X_train

In [15]:
X_train=train_df
X_train

,id,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Balance_nan,Female,Male,France,Spain,Germany,Age_low,Age_high
0,0,15674932,Okwudilichukwu,668,33.0,3,0.00,2,1.0,0.0,181449.97,0,122314.39,False,True,True,False,False,True,False
1,1,15749177,Okwudiliolisa,627,33.0,1,0.00,2,1.0,1.0,49503.50,0,122314.39,False,True,True,False,False,True,False
2,2,15694510,Hsueh,678,40.0,10,0.00,2,1.0,0.0,184866.69,0,122314.39,False,True,True,False,False,True,False
3,3,15741417,Kao,581,34.0,2,148882.54,1,1.0,1.0,84560.88,0,148882.54,False,True,True,False,False,True,False
4,4,15766172,Chiemenam,716,33.0,5,0.00,2,1.0,1.0,15068.83,0,122314.39,False,True,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,Meng,667,33.0,2,0.00,1,1.0,1.0,131834.75,0,122314.39,True,False,False,False,True,True,False
165030,165030,15665521,Okechukwu,792,35.0,3,0.00,1,0.0,0.0,131834.45,0,122314.39,False,True,True,False,False,True,False
165031,165031,15664752,Hsia,565,31.0,5,0.00,1,1.0,1.0,127429.56,0,122314.39,False,True,True,False,False,True,False
165032,165032,15689614,Hsiung,554,30.0,7,161533.00,1,0.0,1.0,71173.03,0,161533.00,True,False,False,False,True,True,False


In [ ]:
X_train.info()

In [16]:
X_train = train_df.drop(["id","Surname","Exited","CustomerId","Balance","Age"],axis=1)
y_train = train_df["Exited"]
X_test  = test_df.drop(["id","Surname","CustomerId","Balance","Age"],axis=1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42,max_depth=5)
model

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_predict = model.predict(X_test)
y_predict[:5]

In [ ]:
feature_names = X_train.columns.tolist()
feature_names

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(30,20))
tree = plot_tree(model,
         feature_names=feature_names,
                 filled=True, fontsize=10)

In [ ]:
model.feature_importances_

In [ ]:
sns.barplot(x=model.feature_importances_,y=feature_names)

In [ ]:
model.score(X_train, Y_train)

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

estimators =[ DecisionTreeClassifier(random_state=42),
              RandomForestClassifier(random_state=42),
              GradientBoostingClassifier(random_state=42)]
estimators

[DecisionTreeClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 GradientBoostingClassifier(random_state=42)]

In [18]:
results = []
for estimator in estimators:
  result = []
  result.append(estimator.__class__.__name__)
  results.append(result)
results

[['DecisionTreeClassifier'],
 ['RandomForestClassifier'],
 ['GradientBoostingClassifier']]

In [19]:
max_depth = np.random.randint(2,20,10)
max_depth

array([10, 15,  2,  4,  7, 15, 13, 18, 12, 13])

In [20]:
max_features = np.random.uniform(0.3,1,10)
max_features

array([0.41442661, 0.56816028, 0.45089751, 0.37842898, 0.63544663,
       0.47896766, 0.48012892, 0.8936324 , 0.47444143, 0.95947949])

In [22]:
from sklearn.model_selection import RandomizedSearchCV
max_depth = np.random.randint(2,20,10)
max_features = np.random.uniform(0.3,1,10)
param_distributions = {"max_depth":max_depth,
                       "max_features":max_features}
results = []
for estimator in estimators:
  result = []
  if estimator.__class__.__name__ !="DecisionTreeClassifier":
     param_distributions["n_estimators"]= np.random.randint(100,200,10)
  clf = RandomizedSearchCV(estimator,
                    param_distributions,
                    n_iter =10,
                    scoring ="accuracy",
                    n_jobs=-1,
                    cv=5,
                    verbose=2 )
  clf.fit(X_train,y_train)
  result.append(estimator.__class__.__name__)
  result.append(clf.score(X_train, y_train))
  result.append(clf.best_params_)
  result.append(clf.best_score_)
  result.append(clf.cv_results_)
  results.append(result)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ........max_depth=7, max_features=0.579170074850526; total time=   0.3s
[CV] END max_depth=3, max_features=0.4329748043835261, n_estimators=126; total time=  11.3s
[CV] END ......max_depth=11, max_features=0.7998442632288527; total time=   0.5s
[CV] END max_depth=7, max_features=0.3545221970228476, n_estimators=173; total time=  18.8s
[CV] END ......max_depth=11, max_features=0.7998442632288527; total time=   0.5s
[CV] END max_depth=7, max_features=0.3545221970228476, n_estimators=173; total time=  18.9s
[CV] END ......max_depth=9, max_features=0.38032904630556735; total time=   0.2s
[CV] END max_depth=7, max_features=0.3545221970228476, n_estimators=173; total time=  18.9s
[CV] END ......max_depth=16, max_features=0.4047373683124642; total time=   0.4s
[CV] END max_depth=7, max_features=0.3545221

In [23]:
df = pd.DataFrame(results,columns=["estimator","test_score","best_params","train_score","cv_results_"])
df

,estimator,test_score,best_params,train_score,cv_results_
0,DecisionTreeClassifier,0.857944,"{'max_features': 0.9361518995025415, 'max_dept...",0.855908,"{'mean_fit_time': [0.3431098461151123, 0.25239..."
1,RandomForestClassifier,0.858575,"{'n_estimators': 129, 'max_features': 0.936151...",0.857508,"{'mean_fit_time': [38.63240561485291, 43.43575..."
2,GradientBoostingClassifier,0.864167,"{'n_estimators': 123, 'max_features': 0.468757...",0.857454,"{'mean_fit_time': [506.4331958770752, 121.2403..."


In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

# 

## 

# DecisionTreeClassifier: 85점